In [243]:
import requests
import pandas as pd
import json

In [2]:
# Collecting data by using APIs
api_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/labs/module%201/Accessing%20Data%20Using%20APIs/jobs.json"
response = requests.get(api_url)# call APIs by using get() 
if response.ok :
    data = response.json() # restore data into json format with dictionary data type

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
type(data)

list

In [26]:
#write Python object into a file as a Json formatted data
with open('job_posting.txt','w') as json_file:
    json.dump(data, json_file) 

In [244]:
# Read Json data from a file and convert it into a dictionary 
with open('job_posting.txt','r') as json_file1:
    job_dataset = json.load(json_file1)

#print(job_dataset)

In [245]:
# load data into a pandas dataframe for data wrangling
dataset1 = pd.DataFrame(job_dataset) # transfer data from json format to dataframe 
#dataset1

In [246]:
# create a function of getting the number of jobs posting for each location
def get_job_number(location):
    t_count = dataset1[dataset1['Location'] == location]
    t_count= t_count[['Job Title']].rename(columns={'Job Title':'The number of jobs posting'}).reset_index()
    return f'The number of jobs posting: {t_count.shape[0]}'


In [247]:
# Call the funtion for the number of jobs posting for each location
t_num = get_job_number(r'Austin')
print(t_num)

The number of jobs posting: 434


In [292]:
# Save the number of jobs posting for each location in an Excel file 
# Using pandas dataframe
t_num.to_excel('Jobs_Posting1.xlsx', sheet_name='Location', index = False)


In [249]:
# list of key skills
k_list = ['C','C#','C++','Java','JavaScript','Python','Scala','Oracle','SQL Server','MySQL Server','PostgreSQL','MongoDB']
#k_list =['Python']

In [250]:
# data wrangling in dataframe
char_dict = {'  ':' ',' | ':'|', ' |':'|', '| ':'|'}
def replace_char(text,dict1):
    for i,j in dict1.items():
        text = text.str.replace(i,j)
    return text

dataset1['Key Skills'] = replace_char(dataset1['Key Skills'], char_dict)
dataset1['Key_Skills_Temp'] = dataset1['Key Skills'].str.lower()


In [251]:
# Update required languages for searching 
k_list1 = ['|C|','C#','C++','|Java|','JavaScript','Python','Scala','Oracle','SQL Server','MySQL Server','PostgreSQL','MongoDB']
dataset1['Key_Skills_String'] = ''

for language in k_list1:
    dataset1['temp'] =-1
    dataset1['temp'] = dataset1["Key_Skills_Temp"].str.find(language.lower())
    dataset1.loc[dataset1['temp']!=-1,'temp'] = language
    dataset1.loc[dataset1['temp']!=-1,'Key_Skills_String'] += language.replace('|','') +","

dataset1.drop(["temp"], axis=1, inplace=True)
dataset1.drop(["Key_Skills_Temp"], axis=1, inplace=True)

In [289]:
# functions of searching jobs posting with required languages
def set_query(languages):
    t_query =''
    flg =1
    languages_list = list(languages.replace(" ","").split(','))
    for lan in languages_list:
        if lan == 'C':
            lan = 'C,'
        elif lan == 'Java':
            lan = ',Java,'
        elif lan == 'C++':
            lan = 'C\+\+'
        elif lan == 'C#':
            lan = 'C\#'
        
        if flg == 1:
            t_query += f'Key_Skills_String.str.contains("{lan}", case=False)'
        else: 
            t_query += f' and Key_Skills_String.str.contains("{lan}", case=False)'
        flg +=1
    return t_query

def search_languages_for_Jobs(languages, dataframe):
    t_query = set_query(languages)
    items= dataframe.query(t_query)   
    print('\033[91m Please enter each language\033[34m after a comma and no space needed, such as\033[35m Python,C#,Java \033[0m')
    return f'The number of jobs posting with languages {languages} is {len(items.index)}'

In [291]:
l_languages= 'c#, python'
search_languages_for_Jobs(l_languages, dataset1)

 Please enter each language after a comma and no space needed, such as Python,C#,Java 


'The number of jobs posting with languages c#, python is 22'